### Text classification with deep learning

Text classification is one of the most basic tasks we can accomplish within nlp, which makes it a great demonstration for the power of some of the important techniques. Here we will discuss processing text a few different ways to feed our text data into a simple neural network model.

To start, we can accomplish building a model purely with our old transformations - in this case, we are going to create binary vectors of the words that represent the IMDB dataset that the first assignment was based on:

In [2]:
from __future__ import print_function

import numpy as np
import keras
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.datasets import imdb

max_features = 1000
batch_size = 32
epochs = 5

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17154048/17464789 [============================>.] - ETA: 0s25000 train sequences
25000 test sequences


In [6]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

Because this is a binary classification (positive or negative reviews), you should get 2 classes back:

In [7]:
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

2 classes


In [8]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (25000, 1000)
x_test shape: (25000, 1000)


In [11]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (25000, 2)
y_test shape: (25000, 2)


In [12]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Building model...
Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 5s - loss: 0.3899 - acc: 0.8253 - val_loss: 0.3495 - val_acc: 0.8568
Epoch 2/5
22500/22500 [==============================] - 4s - loss: 0.3069 - acc: 0.8673 - val_loss: 0.3533 - val_acc: 0.8576
Epoch 3/5
22500/22500 [==============================] - 4s - loss: 0.2620 - acc: 0.8919 - val_loss: 0.3361 - val_acc: 0.8584
Epoch 4/5
22500/22500 [==============================] - 4s - loss: 0.2097 - acc: 0.9176 - val_loss: 0.3598 - val_acc: 0.8544
Epoch 5/5
24160/25000 [===========================>..] - ETA: 0sTest score: 0.359654803686
Test accuracy: 0.8546


## Using embeddings
https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

In [16]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 30
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 12, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[28, 1], [6, 29], [10, 26], [14, 29], [5], [7], [25, 26], [14, 6], [25, 29], [3, 26, 1, 4]]
[[28  1  0  0]
 [ 6 29  0  0]
 [10 26  0  0]
 [14 29  0  0]
 [ 5  0  0  0]
 [ 7  0  0  0]
 [25 26  0  0]
 [14  6  0  0]
 [25 29  0  0]
 [ 3 26  1  4]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4, 12)             360       
_________________________________________________________________
flatten_4 (Flatten)          (None, 48)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 49        
Total params: 409
Trainable params: 409
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 89.999998


### Embeddings built into Keras

In [19]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128, input_length=80))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 18s - loss: 0.4543 - acc: 0.7805 - val_loss: 0.3448 - val_acc: 0.8474
Epoch 2/5
25000/25000 [==============================] - 18s - loss: 0.1750 - acc: 0.9392 - val_loss: 0.3656 - val_acc: 0.8406
Epoch 3/5
25000/25000 [==============================] - 18s - loss: 0.0360 - acc: 0.9959 - val_loss: 0.4039 - val_acc: 0.8389
Epoch 4/5
25000/25000 [==============================] - 18s - loss: 0.0082 - acc: 0.9999 - val_loss: 0.4326 - val_acc: 0.8405
Epoch 5/5
24672/25000 [============================>.] - ETA: 0sTest score: 0.460546208229
Test accuracy: 0.84108


### Exercise: Use downloaded embeddings (either Glove or word2vec)  to build a model to predict the sentiment of a movie review